In [183]:
from os import path
import io
import os
import re
import requests
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from collections.abc import Sequence
from uint import Uint, Int
from enum import Enum, IntEnum
import debugpy as debug
import warnings
from pprint import pprint

In [2]:
from selenium.webdriver import Safari
from selenium import webdriver
from selenium.webdriver.common.by import By
# driver.find_element(By.XPATH, '//button[text()="Some text"]')
# ID = "id"
# XPATH = "xpath"
# LINK_TEXT = "link text"
# PARTIAL_LINK_TEXT = "partial link text"
# NAME = "name"
# TAG_NAME = "tag name"
# CLASS_NAME = "class name"
# CSS_SELECTOR = "css selector"

In [3]:
driver = Safari()
# implicit wait
# driver.manage().timeouts().implicitlyWait(5)
driver.implicitly_wait(10)

In [4]:
driver.get('https://groceries.asda.com')

In [13]:


spiderExplodeDepth = 1

def urlDiscovery(rootUrl:str, driver:Safari) -> list:
    driver.get(rootUrl)
    anchorTagUrls = [we.get_attribute('href') for we in driver.find_elements(By.TAG_NAME, 'a')]
    scriptTagUrlEmbeds = [re.findall(r'^(?:(?:http[s]?|ftp):\/)?\/?(?:[^:\/\s]+)(?:(?:\/\w+)*\/)(?:[\w\-\.]+[^#?\s]+)(?:.*)?(?:#[\w\-]+)?$', we.text) for we in driver.find_elements(By.TAG_NAME, 'script')] 
    onClickAttributeUrlEmbeds = [re.findall(r'^(?:(?:http[s]?|ftp):\/)?\/?(?:[^:\/\s]+)(?:(?:\/\w+)*\/)(?:[\w\-\.]+[^#?\s]+)(?:.*)?(?:#[\w\-]+)?$', we.get_attribute('onClick')) for we in driver.find_elements(By.CSS_SELECTOR, '[onClick]') if we.get_attribute('onClick')] 
    # Button urls should be included in the onClick handler above.
    # buttonTagUrls = [we for we in driver.find_elements(By.TAG_NAME, 'button')]

    urlPioneer = anchorTagUrls + scriptTagUrlEmbeds + onClickAttributeUrlEmbeds
    return urlPioneer

maxSpiderExplodeAllowed = 3
if spiderExplodeDepth > maxSpiderExplodeAllowed:
    warnings.warn(f'Warning: Only run Spider Explode to depth of: {maxSpiderExplodeAllowed}')
for i in range(start=1, stop=max(1,min(spiderExplodeDepth+1,maxSpiderExplodeAllowed+1))):
    urlPioneer = urlDiscovery('https://groceries.asda.com')
    for url in urlPioneer:
        urlPioneer = urlPioneer + urlDiscovery(url)



## Run url discoveries into a path-rank tree

In [190]:
class UrlMatchEnum(IntEnum):
    URL_DOMAIN = 1
    URL_PATH = 2
    URL_QUERY = 11
    URL_QUERY_KEY = 3
    URL_QUERY_VALUE = 4
    URL_PROTOCOL = 5
    URL_PARSING_SYMBOL = 6

In [182]:
class TreeLeaf(object):
    "Generic Tree Leaf"
    def __init__(self, name='root', data=None):
        super().__init__()
        self.name = name
        self.data = data
        if not data:
            self.data = name
    def __repr__(self):
        return self.name

class TreeNode(TreeLeaf):
    "Generic tree node."
    def __init__(self, name='root', data=None, children=None):
        super().__init__(name)
        if data is None:
            self.data = self.name
        else:
            self.data = data
        self._children:list[TreeNode] = []
        self._parent = None
        
        # self._nodeJaggedArr = [[self]]
        if children is not None:
            for child in children:
                self.addChild(child)

    COUNT = [8] # == 2 tabs

    def getChildren(self) -> list:
        return self._children

    children = property(getChildren)

    # Function to print binary tree in 2D
    # It does reverse inorder traversal
    def print2DUtil(root, space:int) -> str:

        output = ''

        # Base case
        if (root == None):
            return ''

        # Increase distance between levels
        space += TreeNode.COUNT[0]

        # Get number of children to root
        n = 0
        if root.children:
            n = len(root.children)

        # print(f'Root {root.data} has {n} kids')
        # print(f'{n} kids -> {(n+1)/2.0} -> {(n+1)/2.0 - 1} -> {int((n+1)/2.0)}')
        # rightKids = ', '.join([str(k.data) for k in root.children[:int((n+1)/2.0):-1]])
        # print(f'Start printing right children from index: {int((n+1)/2.0)}: [{rightKids}]')
        
        # Process right children first
        for childRoot in root.children[:int((n+1)/2.0)-1:-1]:
            output += TreeNode.print2DUtil(childRoot, space)

        # Print current node after space
        # count
        output += '\n' # print()
        for i in range(TreeNode.COUNT[0], space):
            # print(end = " ") # end parameter tells it to append to end of current line rather than create a new line
            output += ' '
        output += f'{root.data}' # print(root.data)

        # Process left child
        for childRoot in root.children[int((n+1)/2.0)-1::-1]:
            output += TreeNode.print2DUtil(childRoot, space)

        return output

    # Wrapper over print2DUtil()
    def print2D(root) :
        
        # space=[0]
        # Pass initial space count as 0
        return TreeNode.print2DUtil(root, 0)

    # A function to do inorder tree traversal
    def traverseInorder(self):

        output = []

        if self:

            # Get number of children to root
            n = 0
            if self.children:
                n = len(self.children)

            # First recur on left child
            for childRoot in self.children[int((n+1)/2.0)-1::-1]:
                output += childRoot.traverseInorder()

            # then print the data of node
            output += [self.data]

            # now recur on right child
            for childRoot in self.children[:int((n+1)/2.0)-1:-1]:
                output += childRoot.traverseInorder()
            
        
        return output


    # A function to do postorder tree traversal
    def traversePostorder(self):

        output = []

        if self:

            # Get number of children to self
            n = 0
            if self.children:
                n = len(self.children)

            # First recur on left child
            for childRoot in self.children[int((n+1)/2.0)-1::-1]:
                output += childRoot.traversePostorder(childRoot)

            # the recur on right child
            for childRoot in self.children[:int((n+1)/2.0)-1:-1]:
                output += childRoot.traversePostorder(childRoot)

            # now print the data of node
            output += [self.data]

        return output

    # A function to do preorder tree traversal
    def traversePreorder(self):

        output = []

        if self:

            # Get number of children to root
            n = 0
            if self.children:
                n = len(self.children)

            # First print the data of node
            output += [self.data]

            # Then recur on left child
            for childRoot in self.children[int((n+1)/2.0)-1::-1]:
                output += childRoot.traversePreorder(childRoot)

            # Finally recur on right child
            for childRoot in self.children[:int((n+1)/2.0)-1:-1]:
                output += childRoot.traversePreorder(childRoot)

        return output
    
    def __repr__(self):
        return TreeNode.print2D(self)

    def __str__(self) -> str:
        return str(self.traverseInorder())

    def __hash__(self) -> int:
        return hash(str(self))

    

    # def getNode(self, level:int, childNo: int):
    #     if self._parent:
    #         return self._parent.getNode(level+1, childNo)
    #     else:
    #         level = max(0, level)
    #         assert level > self.numberOfLayers, f'Thrown trying to get level: {level} of tree which does not exist.'
    #         count = len(self._nodeJaggedArr[level-1])
    #         assert childNo > count, f'Thrown trying to get childno: {childNo} from level: {level} of tree which only has {count} children.'
    #         return self._nodeJaggedArr[level-1][childNo]

    #     assert self.numberOfLayers >= level, f'Tried to get node at level deeper than depth of tree: {self.numberOfLayers}'

        

    
    def getParent(self):
        return self._parent
    
    def setParent(self, parent):
        self._parent = parent
        # del self._nodeJaggedArr

    parent = property(getParent, setParent)
    
    def addChild(self, node):
        assert isinstance(node, TreeNode), 'can only add TreeNode children'
        assert node.parent is None and node.parent != self, f'TreeNode: ({node.name}) can only have one parent. Already has parent with name: {node.parent.name}, so cant set self as parent with name: {self.name}'
        self._children.append(node)
        node.parent = self
        # if self.parent is not None:
        #     self.parent._refreshNodeJaggedArr()
        return self

    # def _refreshNodeJaggedArr(self):
    #     if self.parent is not None:
    #         return self.parent._refreshNodeJaggedArr()
    #     arr = [[self]]
    #     layerInd = 1
        
    #     nextLayer = [True] # Dummy to start loop
    #     i = 1
    #     while nextLayer:
    #         layer = arr[layerInd-1] # [node, node, node]
    #         nextLayer = []
    #         for c in layer:
    #             if c.children:
    #                 nextLayer = nextLayer + c.children
    #         if not nextLayer:
    #             break
    #         else:
    #             arr = arr + [nextLayer]
    #         i += 1

    #     self._nodeJaggedArr = arr

    def getNumberLayers(self):
        # return len(self._nodeJaggedArr) if self._parent is None else (self._parent.getNumberLayers()-1)
        return self.getDepth()

    def getDepth(self) -> int:
        if self.children:
            return max((c.getDepth() for c in self.children))+1
        else:
            return 1


    numberOfLayers = property(getNumberLayers)

    def getLeaves(self):
        _leaves = []
        if not self.children:
            return [self] if not self.parent else [self]
        return [x for c in self.children for x in c.getLeaves()]
            

In [ ]:
import pytest

# def test_tree_can_refresh_jagged_array():
#     thirdChild = TreeNode(f'Leaf{3}')
#     firstChild = TreeNode(f'Leaf{1}', children=[thirdChild])
#     secondChild = TreeNode(f'Leaf{2}')
#     tree = TreeNode(name='TestTree', children=[firstChild, secondChild])
#     tree._refreshNodeJaggedArr()
#     assert tree._nodeJaggedArr == [
#         [tree],
#         [firstChild, secondChild],
#         [thirdChild],
#     ], 'Node Jagged Array has refreshed with the wrong shape'
#     thirdChild._refreshNodeJaggedArr()
#     assert thirdChild._nodeJaggedArr == [
#         [tree],
#         [firstChild, secondChild],
#         [thirdChild],
#     ], 'Node Jagged Array does not correctly refresh from child nodes.'




def test_can_add_child_to_tree():
    tree = TreeNode(name='TestTree')
    tree.addChild(TreeNode('FirstLeaf'))
    assert len(tree.children) == 1, 'TestTree addChild failed to add one child'
    assert isinstance(tree.children[0], TreeNode), 'TestTree addChild doesnt add child of type TreeNode'
    assert tree.children[0].name == 'FirstLeaf', 'TestTree addChild, child incorrectly named'
    assert hasattr(tree.children[0], '_nodeJaggedArr') == False, 'Child TreeNode incorrectly has a _nodeJaggedArray'

def _test_generate_child_node(level:int, childNo: int):
    return TreeNode(name=f'TreeNode_Layer_{level}_child{childNo}')

def test_can_get_tree_leaves():
    tree = TreeNode(name='RootTreeNode')
    numChildren = 2
    layer1 = [_test_generate_child_node(1, i+1) for i in range(numChildren)]
    for i,n in enumerate(layer1):
        tree.addChild(n)
        layer2n = [_test_generate_child_node(2,(i*numChildren)+j+1) for j in range(numChildren)]
        for j,nn in layer2n:
            n.addChild(nn)
    leaves = tree.getLeaves()
    for n in leaves:
        # assert(f'TreeNode_Layer_2_child' in n.name, 'Tree.getLeaves returns nodes that arent leaves')
        assert(not(n.children), 'getLeaves() returns nodes with children')
    assert(len(leaves)==4, 'getLeaves() return incorrect number of leaves in the tree')

def test_tree_can_count_its_layers():
    tree = TreeNode(name='RootTreeNode')
    layer1 = [_test_generate_child_node(1, i+1) for i in range(2)]
    for i,n in enumerate(layer1):
        tree.addChild(n)
        numChildren = 2
        layer2n = [_test_generate_child_node(2,(i*numChildren)+j+1) for j in range(numChildren)]
        for j,nn in layer2n:
            n.addChild(nn)
    assert(tree.numberOfLayers == 3, 'Tree cannot correctly count its layers, should be 3.')
    


def test_can_initialise_tree():
    tree = TreeNode(name='TestTree')
    assert tree.name == 'TestTree', 'Tree node initialisation failed.'


In [199]:
class EncodeUrlRankTreeResult():
    def __init__(self, rankTree, url:str, regexPattern:str) -> None:
        self._tree = rankTree
        self._url = url
        self._re = regexPattern

    def getRankTree(self):
        return self._tree
    
    def getUrl(self):
        return self._url

    def getRegex(self):
        return self._re

class TextRegexPair():
    def __init__(self,text:str,regexPatrn:str) -> None:
        self.text = text
        self.regexPatrn = regexPatrn


class ParsedUrl():
    def __init__(self) -> None:
        self._domain = ''
        self._paths: list[TextRegexPair] = []
        self._queries: list[tuple[TextRegexPair,TextRegexPair]] = []
    
    def getDomain(self):
        return self._domain
    def setDomain(self, domain):
        self._domain = domain
    domain = property(getDomain,setDomain)
    
    def getPaths(self):
        return self._paths
    def addPath(self, path:str, pathRegexPatrn:str):
        self._paths.append(TextRegexPair(path,pathRegexPatrn))
    paths:list[TextRegexPair] = property(getPaths)
    
    def getQueries(self):
        return self._queries
    def addQuery(self, query:tuple, queryRegexPatrn:tuple):
        self._queries.append((TextRegexPair(query[0],queryRegexPatrn[0]),TextRegexPair(query[1],queryRegexPatrn[1])))
    queries:list[tuple[TextRegexPair,TextRegexPair]] = property(getQueries)


class ParsedUrlParser():

    def __init__(self, url:str) -> None:
        self._url = url
        self._fullUrl = f'{url}'
        self._parsedUrl = ParsedUrl()

    captureUrlComponents = False
    requireUrlProtocol = True
    reNoCapt = '' if captureUrlComponents else '?:'
    reCondProtcl = '' if requireUrlProtocol else '?'

    URL_DOMAIN_MATCH = r'^(({0}({0}http[s]?|ftp):\/\/){1}({0}[^:\/\s]+))'.format(reNoCapt, reCondProtcl)

    URL_PATH_MATCH = r'^(({0}\/[\w-]+))'.format(reNoCapt)

    URL_QUERY_FLAG_MATCH = r'^\?'
    
    URL_QUERY_KEY_MATCH = r'^(?:&)?([^\=\&]+)'
    URL_QUERY_VALUE_MATCH = r'^(?:=)([^\=\&]+)'

    def getParsedUrl(self):
        return self._parsedUrl

    parsedUrl:ParsedUrl = property(getParsedUrl)

    def _parseUrlDomain(self):
        
        p = ParsedUrlParser.URL_DOMAIN_MATCH
        _matches = re.findall(p, self._url)
        if _matches:
            assert self._url.index(self._parsedUrl.domain) == 0, 'Domain must be located at index 0'
            self._parsedUrl.domain = _matches[0]
            self._url = self._url[len(self._parsedUrl.domain):]
        assert self._parsedUrl.domain, f'No domain set for url: {self._url}'
        
        return self

    
    def _parseUrlPaths(self):
        assert self._parsedUrl, 'Parsing Url Paths requires a ParsedUrls object'
        assert self._parsedUrl.domain, 'Parsing Url Paths requires the domain to be set on the ParsedUrls object'
        p = ParsedUrlParser.URL_PATH_MATCH
        _matches = re.findall(p, self._url)
        _i = 0
        while _matches and i < 30:
            _i += 1
            m = re.sub(r'^\/', '', _matches[0])
            assert self._url.index(m) == 0, 'Next Path must be located at index 0'
            self._parsedUrl.addPath(m)
            self._url = self._url[len(m):]
            _matches = re.findall(p, self._url)

        return self

    def _parseUrlCheckForQueryFlag(self):
        return bool(re.match(ParsedUrlParser.URL_QUERY_FLAG_MATCH, self._url))

    def _parseUrlQueries(self):
        assert self._parsedUrl, 'Parsing Url Paths requires a ParsedUrls object'
        assert self._parsedUrl.domain, 'Parsing Url Paths requires the domain to be set on the ParsedUrls object'
        
        _matches = re.findall(ParsedUrlParser.URL_QUERY_KEY_MATCH, self._url)
        _i = 0
        while _matches and i < 30:
            _i += 1
            m = _matches[0]
            assert self._url.index(m) == 0, 'Next Path must be located at index 0'
            self._url = self._url[len(m):]
            _qvmatches = re.findall(ParsedUrlParser.URL_QUERY_VALUE_MATCH, self._url)
            if _qvmatches:
                self._parsedUrl.addQuery((m,_qvmatches[0]))
            else:
                self._parsedUrl.addQuery((m,''))
            _matches = re.findall(ParsedUrlParser.URL_QUERY_KEY_MATCH, self._url)

        return self

    def parseUrl(self):
        (self
         ._parseUrlDomain()
         ._parseUrlPaths())
        if self._parseUrlCheckForQueryFlag():
            self._parseUrlQueries()
        return self


In [ ]:

class RankPair():
    def __init__(self, pathFrequencyCounter: Uint, numberRegexNodes: Uint, isRegexNode: bool) -> None:
        self._pathFrequencyCounter: Uint = pathFrequencyCounter
        self._numberRegexNodes: Uint = numberRegexNodes
        self._isRegexNode: bool = isRegexNode

    def getPathFreq(self):
        return self._pathFrequencyCounter

    def incrementPathFreq(self):
        self._pathFrequencyCounter += 1

    def getIsRegexNode(self):
        return self._pathFrequencyCounter

    def getNumRegexNodes(self):
        return self._isRegexNode

    pathFrequencyCounter:Uint = property(getPathFreq)

    isRegexNode:bool = property(getIsRegexNode)

    numberRegexNodes:Uint = property(getNumRegexNodes)

In [203]:
class RankPairTreeNode(TreeNode):
    def __init__(self, name='root', data:RankPair=None, children=None):
        assert isinstance(data, RankPair) or data is None
        super().__init__(name=name, data=data, children=children)
        self.dataAsRankPair: RankPair = data
        self.childrenAsRankPairNodes:list[RankPairTreeNode] = self.children
        self.parentAsRankPairNode:RankPairTreeNode = self.parent

    def getChildrenOfPathType(self):
        return [n for n in self.childrenAsRankPairNodes if not n.dataAsRankPair.isRegexNode]

    def getNumChildrenOfPathType(self):
        return len(self.getChildrenOfPathType())

class RankPairTreeRegexNode(RankPairTreeNode):
    def __init__(self, name='root', data: RankPair = None, children=None):
        super().__init__(name=name, data=data, children=children)

class RankPairTreePathNode(RankPairTreeNode):
    def __init__(self, sisterRegexNode:RankPairTreeRegexNode, name='root', children=None):
        super().__init__(name=name, data=RankPair(1,sisterRegexNode.parentAsRankPairNode.dataAsRankPair.numberRegexNodes,False), children=children)
        self.sisterRegexNode = sisterRegexNode

    def filterPathNodesFromTreeNodes(nodes:list[RankPairTreeNode]):
        result:list[RankPairTreePathNode] = [n for n in nodes if not n.dataAsRankPair.isRegexNode]
        return result
    


In [ ]:

class RankPairTree(object):

    # _rePatternsDecreasingComplexity = [
    #     '(\d+)'
    #     '([^\]+)',
    # ]

    def __init__(self, url:str):
        # super.__init__(self)
        self._urlParser = None
        self._treeState = None
        self.embedUrl(url)

    def embedUrl(self, url:str):
        self._urlParser = ParsedUrlParser(url)
        self._urlParser.parseUrl()

        if self._treeState is None:
            self._treeState = RankPairTreeNode(
                name=self._urlParser.parsedUrl.domain,
                data=RankPair(1,0,False)
                )
        elif self._treeState.name != self._urlParser.parsedUrl.domain:
            self._treeState = RankPairTree(
                name=RankPairTree.TREE_ROOT_MULTI_DOMAIN_NAME,
                data=RankPair(pathFrequencyCounter=1, numberRegexNodes=0, isRegexNode=False), 
                children=[
                    self._treeState,
                    RankPairTree(self._urlParser.parsedUrl.domain)
                ])
        
        nodesToAddTo = [self.getDomainNode()]

        # get child nodes if domain has already been embedded with other urls
        nodesToAddTo = [childNode for node in nodesToAddTo for childNode in node.children]

        def _f(p:str,regx:str, nodesToAddTo:list[RankPairTreeNode]):
            '''We are only adding text and regex nodes on all leaves when it is a new domain tree.\n
            Otherwise we only add paths and potentially regex when the tree does NOT already contain them.'''
            pathNodes = RankPairTreePathNode.filterPathNodesFromTreeNodes(nodesToAddTo)
            matchingPathLeaves = [x for n in pathNodes if n.name == p for x in [n,n.sisterRegexNode]]
            if matchingPathLeaves:
                nodesToAddTo = matchingPathLeaves
            else:
                reLeaves = [lrg for lrg in nodesToAddTo if lrg.dataAsRankPair.isRegexNode and re.match(lrg.name, p)]
                if reLeaves:
                    nodesToAddTo = []
                    for rgxLeaf in reLeaves:
                        # add new path node to rgxLeaf.parent
                        pathnode = RankPairTreePathNode(sisterRegexNode=rgxLeaf,name=p) #link the path node to regexLeaf
                        rgxLeaf.parent.addChild(pathnode) 
                        rgxLeaf.dataAsRankPair.incrementPathFreq()
                        nodesToAddTo.append(pathnode)
                        nodesToAddTo.append(rgxLeaf)
                else:
                    newNodes = []
                    for node in nodesToAddTo:
                        self._addTextAndRegexNodes(node, p, regx)
                        pathRegexNode = RankPairTreeRegexNode(name=(RankPairTree._getSubRePattern(p) or regx), 
                                                              data=RankPair(pathFrequencyCounter=node.getNumChildrenOfPathType(), 
                                                                            numberRegexNodes=node.dataAsRankPair.numberRegexNodes + 1))
                        pathTextNode = RankPairTreePathNode(sisterRegexNode=pathRegexNode, name=p)
                        (node
                        .addChild(pathTextNode)
                        .addChild(pathRegexNode))
                        newNodes.append(pathTextNode)
                        newNodes.append(pathRegexNode)
                    nodesToAddTo = newNodes
            return nodesToAddTo
        
        for p,regx in self._urlParser.parsedUrl.paths:
            nodesToAddTo = _f(p,regx,nodesToAddTo)
            

        for (qk, qkRgx),(qv, qvRgx) in self._urlParser.parsedUrl.queries:
            nodesToAddTo = _f(qk, qkRgx, nodesToAddTo)
            if qv:
                nodesToAddTo = _f(qv, qvRgx, nodesToAddTo)

        return self


    def __repr__(self) -> str:
        return self._treeState.__repr__()
    
    def __str__(self) -> str:
        return self._treeState.__str__()
    
    def getChildren(self):
        return self._treeState.children

    children: list[RankPairTreeNode] = property(getChildren)

    def addChild(self, node:RankPairTreeNode):
        leaves = self.getLeaves()
        for leaf in leaves:
            leaf.addChild(node)
        return self
    
    def getDomainNode(self):
        domain = self._urlParser.parsedUrl.domain
        if self._treeState.name == domain:
            return self._treeState
        elif self._treeState.name == RankPairTree.TREE_ROOT_MULTI_DOMAIN_NAME:
            domainNode = [n for n in self._treeState.children if n.name == domain]
            if domainNode:
                return domainNode[0]
        else:
            return None

    def getLeaves(self) -> list[RankPairTreeNode]:
        domain = self._urlParser.parsedUrl.domain
        if self._treeState.name == domain:
            return self._treeState.getLeaves()
        elif self._treeState.name == RankPairTree.TREE_ROOT_MULTI_DOMAIN_NAME:
            domainNode = [n for n in self._treeState.children if n.name == domain]
            if domainNode:
                return domainNode[0].getLeaves()
        else:
            return []

    def getLeavesByRegx(self) -> list[RankPairTreeNode]:
        return self.getLeaves()    
    
    def getDepth(self):
        return self._treeState.numberOfLayers
    
    depth = property(getDepth)

    TREE_ROOT_DEFAULT_NAME = 'root'

    TREE_ROOT_MULTI_DOMAIN_NAME = 'MULTI_DOMAIN'

    def _getSubRePattern(testStr:str) -> str:
        '''
        decreasing order of specificity rules: ['[0-9]', '[A-Z]', '[A-Za-z]', '[0-9A-Za-z]', r'[0-9A-Za-z\-]', r'[^\/]', '', '', '', ]
        TODO: Make sure remove the leading / from path matches
        to match: ['forum', 'p', 'sid', '0193Q']
        '''
        tryPatternsDecreasingSpecificity = [
            '^[0-9]+$',
            '^[A-Z]+$',
            '^[A-Za-z]+$',
            '^[0-9A-Za-z]+$',
            r'^[0-9A-Za-z\-]+$',
            r'^[^\/]+$',
            r'^[^=]+$',
        ]

        for pattern in tryPatternsDecreasingSpecificity:
            if re.match(pattern, testStr):
                return pattern
        else:
            return None
    

    def _encodeUrlDomain(rankTree, url:str):
        if not rankTree:
            return
        p = RankPairTree.URL_DOMAIN_MATCH
        _matches = re.findall(p, url)
        if _matches:
            _matchedStr = _matches[0]
            if rankTree.name == RankPairTree.TREE_ROOT_DEFAULT_NAME:
                rankTree = RankPairTree(_matchedStr)
                return rankTree
            elif rankTree.name == _matchedStr:
                return rankTree
            else:
                return RankPairTree(
                    name=RankPairTree.TREE_ROOT_MULTI_DOMAIN_NAME,
                    data=RankPair(pathFrequencyCounter=1, numberRegexNodes=0, isRegexNode=False), 
                    children=[
                        rankTree,
                        RankPairTree(_matchedStr)
                    ])
        else:
            return None
            

        return EncodeUrlRankTreeResult(rankTree, url, _matchedStr)


    def _encodeUrlNextPath(rankTree, url:str):
        if not rankTree:
            return
        p = RankPairTree.URL_PATH_MATCH
        _matches = re.findall(p, url)
        if _matches:
            _matchedStr = re.sub(r'^\/', '', _matches[0])

    def _addTextAndRegexNodes(self, node:RankPairTreeNode, text:str, regexPtrn:str):
        pathRegexNode = RankPairTreeRegexNode(name=(RankPairTree._getSubRePattern(text) or regexPtrn), 
                                              data=RankPair(pathFrequencyCounter=node.getNumChildrenOfPathType(), 
                                                            numberRegexNodes=node.dataAsRankPair.numberRegexNodes + 1))
        pathTextNode = RankPairTreePathNode(sisterRegexNode=pathRegexNode, name=text)
        (node
        .addChild(pathTextNode)
        .addChild(pathRegexNode))


    def encodeUrlToNewRankPairTree(url:str, rankTree=None):
        _testUrl = f'{url}'
        startingUrl = f'{_testUrl}'
        if rankTree is None:
            rankTree = RankPairTree(name=RankPairTree.TREE_ROOT_DEFAULT_NAME)


            
            
            pList = RankPairTree._patterns[UrlMatchEnum.URL_DOMAIN]
            nodes = rankTree.getLeaves()
            for pList in RankPairTree._patterns:
                _i = -1
                
                someMatchesInpList = True
                while someMatchesInpList and _i < 99:
                    someMatchesInpList = False
                    for (p, url_match_part) in pList:
                        _i += 1
                        _matches = re.findall(p, _testUrl)
                        if _matches:
                            _matchedStr = _matches[0]
                            if url_match_part == UrlMatchEnum.URL_PATH:
                                _matchedStr = re.sub(r'^\/', '', _matches[0])
                            
                            #TODO:? is this line needed ->  _matches = re.findall(p, _testUrl)
                            someMatchesInpList = True

                            RankPairTree._checkDomainMatches(rankTree, _testUrl)

                            
                            if url_match_part == UrlMatchEnum.URL_DOMAIN:
                                rankTree = RankPairTree(_matchedStr)
                                nodes = rankTree.getLeaves()
                            elif url_match_part == UrlMatchEnum.URL_PATH or url_match_part == UrlMatchEnum.URL_QUERY_KEY or url_match_part == UrlMatchEnum.URL_QUERY_VALUE:
                                for node in nodes:
                                    pathTextNode = RankPairTreeNode(name=_matchedStr,
                                                                    data=RankPair(pathFrequencyCounter=node.getNumChildrenOfPathType()+1, 
                                                                                 numberRegexNodes=node.data.numberRegexNodes))
                                    pathRegexNode = RankPairTreeNode(name=(RankPairTree._getSubRePattern(_matchedStr) or p), 
                                                                    data=RankPair(pathFrequencyCounter=node.getNumChildrenOfPathType(), 
                                                                                 numberRegexNodes=node.data.numberRegexNodes + 1))
                                    (node
                                    .addChild(pathTextNode)
                                    .addChild(pathRegexNode))

                            nodes = rankTree.getLeaves()
        # print(rankTree.depth)
        # pprint(rankTree)
        # print(rankTree.getLeaves())
        return rankTree

    
    def encodeUrlToNewRankPairTreeOld(url:str, rankTree=None):
        _testUrl = f'{url}'
        startingUrl = f'{_testUrl}'
        if rankTree is None:
            rankTree = RankPairTree(name='root')

            nodes = rankTree.getLeaves()

            
            
            pList = RankPairTree._patterns[UrlMatchEnum.URL_DOMAIN]
            for pList in RankPairTree._patterns:
                _i = -1
                
                someMatchesInpList = True
                while someMatchesInpList and _i < 99:
                    someMatchesInpList = False
                    for (p, url_match_part) in pList:
                        _i += 1
                        _matches = re.findall(p, _testUrl)
                        if _matches:
                            _matchedStr = _matches[0]
                            if url_match_part == UrlMatchEnum.URL_PATH:
                                _matchedStr = re.sub(r'^\/', '', _matches[0])
                            
                            #TODO:? is this line needed ->  _matches = re.findall(p, _testUrl)
                            someMatchesInpList = True

                            RankPairTree._checkDomainMatches(rankTree, _testUrl)

                            
                            if url_match_part == UrlMatchEnum.URL_DOMAIN:
                                rankTree = RankPairTree(_matchedStr)
                                nodes = rankTree.getLeaves()
                            elif url_match_part == UrlMatchEnum.URL_PATH or url_match_part == UrlMatchEnum.URL_QUERY_KEY or url_match_part == UrlMatchEnum.URL_QUERY_VALUE:
                                for node in nodes:
                                    pathTextNode = RankPairTreeNode(name=_matchedStr,
                                                                    data=RankPair(pathFrequencyCounter=node.getNumChildrenOfPathType()+1, 
                                                                                numberRegexNodes=node.data.numberRegexNodes))
                                    pathRegexNode = RankPairTreeNode(name=(RankPairTree._getSubRePattern(_matchedStr) or p), 
                                                                    data=RankPair(pathFrequencyCounter=node.getNumChildrenOfPathType(), 
                                                                                numberRegexNodes=node.data.numberRegexNodes + 1))
                                    (node
                                    .addChild(pathTextNode)
                                    .addChild(pathRegexNode))

                            nodes = rankTree.getLeaves()
        # print(rankTree.depth)
        # pprint(rankTree)
        # print(rankTree.getLeaves())
        return rankTree

    

    def _splitPathAndQuery(self, url:str) -> tuple[str,str]:
        pattern = r'^((?:(?:http[s]?|ftp):\/)?\/?(?:[^:\/\s]+)(?:(?:\/\w+)*\/)(?:[\w\-\.]+[^#?\s]+))\?((?:[\w\d]+=[\w\d]+&?)+)(?:.*)(?:#[\w\-]+)?$'
        pathQrySplit = re.findall(pattern, url)
        if pathQrySplit:
            return pathQrySplit[0]
        else:
            return None

    def _findQueryRegexPattern(self, url:str) -> str:
        pass
    
    def _findPathRegexPattern(self, url:str) -> str:
        pass

    def _findRegexPatern(self, url:str) -> str:
        pass

    
        



In [194]:
'hello'.index('el')

1

In [17]:
re.findall(r'^((?:(?:http[s]?|ftp):\/)?\/?(?:[^:\/\s]+)(?:(?:\/\w+)*\/)(?:[\w\-\.]+[^#?\s]+))\?((?:[\w\d]+=[\w\d]+&?)+)(?:.*)(?:#[\w\-]+)?$',
    'https://groceries.asda.com/product/natural-plain-organic/fage-total-fat-free-greek-recipe-natural-yogurt/24771357?sid=12534Q&style=green')

[('https://groceries.asda.com/product/natural-plain-organic/fage-total-fat-free-greek-recipe-natural-yogurt/24771357',
  'sid=12534Q&style=green')]

In [181]:

import pytest
def rest_rankTree_builds_correct_layers():
    
    _testUrl = 'https://groceries.asda.com/product/natural-plain-organic/fage-total-fat-free-greek-recipe-natural-yogurt/24771357?sid=12534Q&style=green'
    rankTree = RankPairTree.encodeUrlToRankPairTree(_testUrl)
    assert rankTree.depth == 9, f'RankTree should have 9 layers for url: {startingUrl}'
    pprint(rankTree)

In [ ]:
from selenium.webdriver import Safari
from selenium import webdriver
from selenium.webdriver.common.by import By
# driver.find_element(By.XPATH, '//button[text()="Some text"]')
# ID = "id"
# XPATH = "xpath"
# LINK_TEXT = "link text"
# PARTIAL_LINK_TEXT = "partial link text"
# NAME = "name"
# TAG_NAME = "tag name"
# CLASS_NAME = "class name"
# CSS_SELECTOR = "css selector"

In [ ]:
driver = Safari()
# implicit wait
# driver.manage().timeouts().implicitlyWait(5)
driver.implicitly_wait(10)

[{}]

In [ ]:
# urlClusterState = set()
tree = RankPairTree()

for url in urlPioneer:
    tree.encodeUrlToRankPairTree(url)